In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/02/24 11:53:41 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
# !wget https://github.com/DataTalksClub/nyc-tlc-data/releases/download/fhvhv/fhvhv_tripdata_2021-01.csv.gz

In [4]:
# !gzip -d fhvhv_tripdata_2021-01.csv.gz

In [5]:
!wc -l fhvhv_tripdata_2021-01.csv

11908469 fhvhv_tripdata_2021-01.csv


In [6]:
 df = spark.read \
    .option('header', 'true') \
    .csv('fhvhv_tripdata_2021-01.csv')

In [7]:
df.head(5)

[Row(hvfhs_license_num='HV0003', dispatching_base_num='B02682', pickup_datetime='2021-01-01 00:33:44', dropoff_datetime='2021-01-01 00:49:07', PULocationID='230', DOLocationID='166', SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02682', pickup_datetime='2021-01-01 00:55:19', dropoff_datetime='2021-01-01 01:18:21', PULocationID='152', DOLocationID='167', SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_datetime='2021-01-01 00:23:56', dropoff_datetime='2021-01-01 00:38:05', PULocationID='233', DOLocationID='142', SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_datetime='2021-01-01 00:42:51', dropoff_datetime='2021-01-01 00:45:50', PULocationID='142', DOLocationID='143', SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_datetime='2021-01-01 00:48:14', dropoff_datetime='2021-01-01 01:08:42', PULocationID='143', DOLocationID='78', SR_Flag=None)]

In [8]:
df.schema

StructType([StructField('hvfhs_license_num', StringType(), True), StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', StringType(), True), StructField('dropoff_datetime', StringType(), True), StructField('PULocationID', StringType(), True), StructField('DOLocationID', StringType(), True), StructField('SR_Flag', StringType(), True)])

In [9]:
!head -n 101 fhvhv_tripdata_2021-01.csv > head.csv

In [10]:
import pandas as pd

In [11]:
df_pandas = pd.read_csv('head.csv')

In [12]:
df_pandas.dtypes

hvfhs_license_num        object
dispatching_base_num     object
pickup_datetime          object
dropoff_datetime         object
PULocationID              int64
DOLocationID              int64
SR_Flag                 float64
dtype: object

In [13]:
spark.createDataFrame(df_pandas).schema

StructType([StructField('hvfhs_license_num', StringType(), True), StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', StringType(), True), StructField('dropoff_datetime', StringType(), True), StructField('PULocationID', LongType(), True), StructField('DOLocationID', LongType(), True), StructField('SR_Flag', DoubleType(), True)])

In [14]:
from pyspark.sql import types

In [23]:
schema = types.StructType([
    types.StructField('hvfhs_license_num', types.StringType(), True),
    types.StructField('dispatching_base_num', types.StringType(), True),
    types.StructField('pickup_datetime', types.TimestampType(), True),
    types.StructField('dropoff_datetime', types.TimestampType(), True),
    types.StructField('PULocationID', types.IntegerType(), True),
    types.StructField('DOLocationID', types.IntegerType(), True),
    types.StructField('SR_Flag', types.StringType(), True)
])

In [24]:
 df = spark.read \
    .option('header', 'true') \
    .schema(schema) \
    .csv('fhvhv_tripdata_2021-01.csv')

In [25]:
df.repartition(24)

DataFrame[hvfhs_license_num: string, dispatching_base_num: string, pickup_datetime: timestamp, dropoff_datetime: timestamp, PULocationID: int, DOLocationID: int, SR_Flag: string]

In [27]:
df.write.parquet('fhvhv/2021/01/', mode='overwrite')

In [28]:
df = spark.read.parquet('fhvhv/2021/01/')

In [29]:
df.printSchema()

root
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- SR_Flag: string (nullable = true)



In [32]:
df.select('pickup_datetime', 'dropoff_datetime', 'PULocationID', 'DOLocationID')\
    .filter(df.hvfhs_license_num == 'HV0003') \
    .show()

+-------------------+-------------------+------------+------------+
|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|
+-------------------+-------------------+------------+------------+
|2021-01-18 09:26:34|2021-01-18 09:45:13|         144|         181|
|2021-01-18 09:01:10|2021-01-18 09:19:18|         145|         138|
|2021-01-18 09:23:03|2021-01-18 09:28:40|         138|          70|
|2021-01-18 09:34:22|2021-01-18 09:40:02|         138|         138|
|2021-01-18 09:03:11|2021-01-18 09:10:50|          92|         171|
|2021-01-18 09:39:57|2021-01-18 09:53:46|          53|         213|
|2021-01-18 09:50:14|2021-01-18 10:02:09|          42|         119|
|2021-01-18 09:59:00|2021-01-18 10:03:01|         153|         220|
|2021-01-18 09:14:15|2021-01-18 09:34:47|           7|         237|
|2021-01-18 09:05:00|2021-01-18 09:22:26|         119|         166|
|2021-01-18 09:02:52|2021-01-18 09:20:40|         256|          17|
|2021-01-18 09:30:05|2021-01-18 09:39:59|       

In [34]:
df.select('pickup_datetime', 'dropoff_datetime', 'PULocationID', 'DOLocationID')\
    .filter(df.hvfhs_license_num == 'HV0003') \
    .take(5)

[Row(pickup_datetime=datetime.datetime(2021, 1, 18, 9, 26, 34), dropoff_datetime=datetime.datetime(2021, 1, 18, 9, 45, 13), PULocationID=144, DOLocationID=181),
 Row(pickup_datetime=datetime.datetime(2021, 1, 18, 9, 1, 10), dropoff_datetime=datetime.datetime(2021, 1, 18, 9, 19, 18), PULocationID=145, DOLocationID=138),
 Row(pickup_datetime=datetime.datetime(2021, 1, 18, 9, 23, 3), dropoff_datetime=datetime.datetime(2021, 1, 18, 9, 28, 40), PULocationID=138, DOLocationID=70),
 Row(pickup_datetime=datetime.datetime(2021, 1, 18, 9, 34, 22), dropoff_datetime=datetime.datetime(2021, 1, 18, 9, 40, 2), PULocationID=138, DOLocationID=138),
 Row(pickup_datetime=datetime.datetime(2021, 1, 18, 9, 3, 11), dropoff_datetime=datetime.datetime(2021, 1, 18, 9, 10, 50), PULocationID=92, DOLocationID=171)]

In [35]:
from pyspark.sql import functions as F

In [42]:
def crazy_stuff(base_num):
    num = int(base_num[1:])
    if num % 7 == 0:
        return f's/{num:03x}'
    if num % 3 == 0:
        return f'a/{num:03x}'
    else:
        return f'e/{num:03x}'

In [43]:
crazy_stuff('B02884')

's/b44'

In [44]:
crazy_studd_udf = F.udf(crazy_stuff, returnType=types.StringType())

In [46]:
df \
    .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
    .withColumn('dropoff_date', F.to_date(df.dropoff_datetime)) \
    .withColumn('base_id', crazy_studd_udf(df.dispatching_base_num)) \
    .select('base_id', 'pickup_date', 'dropoff_date', 'PULocationID', 'DOLocationID') \
    .show()

+-------+-----------+------------+------------+------------+
|base_id|pickup_date|dropoff_date|PULocationID|DOLocationID|
+-------+-----------+------------+------------+------------+
|  s/b3d| 2021-01-18|  2021-01-18|         144|         181|
|  e/b3b| 2021-01-18|  2021-01-18|         145|         138|
|  e/b3b| 2021-01-18|  2021-01-18|         138|          70|
|  e/b3b| 2021-01-18|  2021-01-18|         138|         138|
|  s/b44| 2021-01-18|  2021-01-18|          92|         171|
|  s/b44| 2021-01-18|  2021-01-18|          53|         213|
|  e/acc| 2021-01-18|  2021-01-18|          42|         119|
|  s/b44| 2021-01-18|  2021-01-18|         153|         220|
|  e/acc| 2021-01-18|  2021-01-18|           7|         237|
|  e/9ce| 2021-01-18|  2021-01-18|         132|         263|
|  e/b38| 2021-01-18|  2021-01-18|         119|         166|
|  s/b44| 2021-01-18|  2021-01-18|         256|          17|
|  s/b44| 2021-01-18|  2021-01-18|         256|         112|
|  s/b44| 2021-01-18|  2